<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/jangjaewon/20220917_codetest(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## code test

In [2]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys

In [3]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab

Mounted at /content/gdrive/


In [4]:
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

['CHANGELOG.txt', 'LICENSE.txt', 'SHA256SUMS.txt', 'index.html', 'icu', 'hosp']

In [5]:
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv')
admission

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454319,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
454320,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
454321,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
454322,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [16]:
# labevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/labevents.csv')
# patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv')
# d_labitems = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/d_labitems.csv')
# diagnosis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnosis.csv')


In [17]:
import sqlite3

In [18]:

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

In [19]:
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [ ]:
# Step 1: Read the csv file into a dataframe
# Dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress
input_df = pd.read_csv('country_vaccinations.csv')

# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='country_vaccinations',
            db_name='default.db')

# Step 3: Write the SQL query in a string variable
sql_query_string = """
    SELECT country, SUM(daily_vaccinations) as total_vaccinated
    FROM country_vaccinations 
    WHERE daily_vaccinations IS NOT NULL 
    GROUP BY country
    ORDER BY total_vaccinated DESC
"""

# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [36]:
input_df = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv')

In [21]:
pd_to_sqlDB(input_df,
            table_name='admissions',
            db_name='default.db')


In [23]:
sql_query_string = """
    SELECT count(distinct subject_id)
    FROM admissions 
"""

In [24]:
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,count(distinct subject_id)
0,190279


In [22]:
input_df.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [31]:
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv')
pd_to_sqlDB(patients,
            table_name='patients',
            db_name='default.db')

In [29]:
patients.columns

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')

In [32]:
sql_query_string = """
    SELECT count(distinct subject_id)
    FROM patients 
"""
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,count(distinct subject_id)
0,315460


In [60]:
diagnoses_icd = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnoses_icd.csv')
pd_to_sqlDB(diagnoses_icd,
            table_name='diagnoses_icd',
            db_name='default.db')

In [40]:
diagnoses_icd.columns

Index(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version'], dtype='object')

In [48]:
diagnoses_icd['icd_code'].head()

0     5723
1    78959
2     5715
3    07070
4      496
Name: icd_code, dtype: object

In [50]:
diagnoses_icd['icd_version'].head()

0    9
1    9
2    9
3    9
4    9
Name: icd_version, dtype: int64

In [55]:
np.sum(diagnoses_icd['icd_code']=='99591') 

2964

In [62]:
np.sum(diagnoses_icd['icd_code']=='A419') 

4757

In [56]:
np.sum(diagnoses_icd['icd_version']==9)

2915548

In [ ]:
#ICU 입실한 환자
#24시간 이내 사망한 환자 제외
#24시간 이내 lab
#spesis 뿐만 아니라 infection 코드가 붙어 있는 사람
sql_query = """ 
create table subject_id_2 as (
  select count(distinct subject_id)
  from diagnoses_icd
  where (icd_version = 9 and icd_code = '99591') or (icd_version = 10 and icd_code = 'A419')
);
create table subject_id_1 as (
  select 

);

"""

result_df = sql_query_to_pd(sql_query, db_name='default.db')
result_df

In [63]:
#ICU 입실한 환자
#24시간 이내 사망한 환자 제외
#24시간 이내 lab
#spesis 뿐만 아니라 infection 코드가 붙어 있는 사람
sql_query = """ 
create table subject_id_1 as (
  select subject_id
  from diagnoses_icd
  where (icd_version = 9 and icd_code = '99591') or (icd_version = 10 and icd_code = 'A419')
);
create table subject_id_2 as (
  select 
  from 
  where 
);

"""

result_df = sql_query_to_pd(sql_query, db_name='default.db')
result_df

,count(distinct subject_id)
0,6759


In [7]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
%load_ext google.colab.data_table

In [12]:
from google.cloud import bigquery

In [13]:
client = bigquery.Client(project='4/0ARtbsJrRvmKJ9YdfG36DUkkIECN4lYSEudwbqY3VilWte6bBI2a8tD3hNAg1B0MPdPOrIQ')

In [14]:
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count

SyntaxError: ignored

# 새 섹션